In [267]:
import pandas as pd 
import random
df = pd.read_csv('https://raw.githubusercontent.com/m3gaq/finodays_mkb/main/table_v2.zip')
df.columns

Index(['clientcode', 'time_stamp_start', 'useragent_start', 'url_start',
       'url_end', 'time_stamp_end', 'useragent_end', 'diff', 'reload',
       'start_flag', 'start_flag_num', 'process_flag_num', 'start_product'],
      dtype='object')

In [188]:
df['clientcode'] = df['clientcode'].astype(int)
randomlist = random.sample(list(df['clientcode']), 100)
df_random = df[df['clientcode'].isin(randomlist)]

In [189]:
sources  = list(df_random.groupby(['url_start','url_end']).size().reset_index().rename(columns={0:'cnt_open'})['url_start'])
targets = list(df_random.groupby(['url_start','url_end']).size().reset_index().rename(columns={0:'cnt_open'})['url_end'])
cnt_open = list(df_random.groupby(['url_start','url_end']).size().reset_index().rename(columns={0:'cnt_open'})['cnt_open'])
unique_pages = list(set(sources + targets))

In [190]:
numbers = range(0, len(unique_pages))
d = dict(zip(unique_pages, numbers))
df_random['source_id'] = df_random['url_start'].map(d)
df_random['target_id'] = df_random['url_end'].map(d)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [191]:
incoming = []
outcoming = []
for i in unique_pages:
  a = sources.count(i)
  b = targets.count(i)
  incoming.append(a)
  outcoming.append(b)

data = {'Pages': unique_pages,
        'Incoming': incoming, 
        'Outcoming': outcoming }
df_new = pd.DataFrame(data)
pages_bad= list(df_new[df_new['Incoming'] == df_new['Outcoming']]['Pages'])

In [192]:
import numpy as np
df_random['GB1'] = np.where(df_random['url_start'].isin(pages_bad), 'bad', 'good')
df_random['GB2'] = df_random['GB1'].shift(-1)
df_random['url_end_shift'] = df_random['url_end'].shift(-1)
df_random['url_end_copy'] = np.where(df_random['GB2'] == 'bad', df_random['url_end_shift'], df_random['url_end'])
df_random = df_random[df_random['GB1'] != 'bad']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = list(d.keys()),
      color = "blue"
    ),
    link = dict(
      source = list(df_random['source_id']),
      target = list(df_random['target_id']),
      value = cnt_open
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [263]:
def get_metrics(df):
    '''
    return DataFrames:
    metrics_node, metrics_relation 
    '''
    df['time_stamp_start'] = pd.to_datetime(df.time_stamp_start)
    df['time_stamp_end']   = pd.to_datetime(df.time_stamp_end)

    metrics_relation = df.groupby(['url_start','url_end']).agg({'diff':['mean','max','min','sum','count'],'reload':['mean'], 'process_flag_num':['mean']})
    metrics_relation.columns = [ '_'.join(col) for col in metrics_relation.columns]
    metrics_relation = metrics_relation.rename(columns={'diff_count':'qty_rel_all','process_flag_num_mean':'process_order_mean','reload_mean':'is_reload'})
    metrics_relation = metrics_relation.reset_index()

    qty_all = df.groupby('url_start').count().rename(columns={'clientcode':'qty_all'})[['qty_all']]
    qty_unique_user = df[['clientcode','url_start']].drop_duplicates().groupby('url_start').count().rename(columns={'clientcode':'qty_unique_user'})
    qty_in = metrics_relation.reset_index().groupby('url_start').sum().rename(columns = {'qty_rel_all':'qty'})
    qty_in.columns = qty_in.columns+'_in'
    qty_out = metrics_relation.reset_index().groupby('url_end').sum().rename(columns = {'qty_rel_all':'qty'})
    qty_out.columns = qty_out.columns+'_out'

    metrics_node = pd.merge(qty_all,qty_unique_user,left_index=True, right_index=True, how = 'left')
    metrics_node = pd.merge(metrics_node,qty_in,left_index=True, right_index=True, how = 'left')
    metrics_node = pd.merge(metrics_node,qty_out,left_index=True, right_index=True, how = 'left')
    metrics_node['qty_in_out_ratio'] = metrics_node.qty_in/metrics_node.qty_out
    metrics_node = metrics_node.reset_index()

    return metrics_node, metrics_relation

In [269]:
df_random_copy = df_random.copy()
df_random_copy['time_stamp_start'] = pd.to_datetime(df_random_copy.time_stamp_start)
df_random_copy['time_stamp_end']   = pd.to_datetime(df_random_copy.time_stamp_end)
df = df_random_copy.copy()
metrics_relation = df.groupby(['url_start','url_end']).agg({'diff':['mean','max','min','sum','count'],'reload':['mean'], 'process_flag_num':['mean']})
metrics_relation.columns = [ '_'.join(col) for col in metrics_relation.columns]
metrics_relation = metrics_relation.rename(columns={'diff_count':'qty_rel_all','process_flag_num_mean':'process_order_mean','reload_mean':'is_reload'})
metrics_relation = metrics_relation.reset_index()
qty_all = df.groupby('url_start').count().rename(columns={'clientcode':'qty_all'})[['qty_all']]
qty_unique_user = df[['clientcode','url_start']].drop_duplicates().groupby('url_start').count().rename(columns={'clientcode':'qty_unique_user'})
qty_in = metrics_relation.reset_index().groupby('url_start').sum().rename(columns = {'qty_rel_all':'qty'})
qty_in.columns = qty_in.columns+'_in'
qty_out = metrics_relation.reset_index().groupby('url_end').sum().rename(columns = {'qty_rel_all':'qty'})
qty_out.columns = qty_out.columns+'_out'
metrics_node = pd.merge(qty_all,qty_unique_user,left_index=True, right_index=True, how = 'left')
metrics_node = pd.merge(metrics_node,qty_in,left_index=True, right_index=True, how = 'left')
metrics_node = pd.merge(metrics_node,qty_out,left_index=True, right_index=True, how = 'left')
metrics_node['qty_in_out_ratio'] = metrics_node.qty_in/metrics_node.qty_out
metrics_node = metrics_node.reset_index()
cnt_metric = metrics_relation['diff_mean']
fig = go.Figure(data=[go.Sankey(
      node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = list(d.keys()),
        color = "blue"
      ),
      link = dict(
        source = list(df_random['source_id']),
        target = list(df_random['target_id']),
        value = cnt_metric
    ))])
fig.show()

In [23]:
table_metrics

,url_start,url_end,diff_mean,diff_max,diff_min,diff_sum,qty_rel_all,is_reload,process_order_mean
0,accounts,dcards,5.000000,6,4,10,2,0.0,23.500000
1,accounts,deps,44.500000,59,30,89,2,0.0,12.500000
2,accounts,main,39.000000,39,39,39,1,0.0,4.000000
3,accounts,paymentform,17.000000,20,15,51,3,0.0,14.000000
4,autopayments/list,paymentservices/create,8.000000,8,8,8,1,0.0,2.000000
5,bonusprogram/mkbbonus/compensate/confirmation,complete,8.000000,8,8,8,1,0.0,16.000000
6,bonusprogram/mkbbonus/compensate/create,bonusprogram/mkbbonus/compensate/confirmation,13.000000,13,13,13,1,0.0,15.000000
7,bonusprogram/mkbbonus/compensate/create,bonusprogram/mkbbonus/compensate/create,3.000000,6,1,9,3,1.0,13.000000
8,bonusprogram/mkbbonus/compensate/create,bonusprogram/mkbbonus/main,30.666667,67,7,92,3,0.0,23.666667
9,bonusprogram/mkbbonus/compensate/create,main,746.000000,746,746,746,1,0.0,9.000000


In [30]:
import pandas as pd 
import random
import plotly.graph_objects as go
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/m3gaq/finodays_mkb/main/data/table_v4.zip')

def get_metrics(df):
    '''
    return DataFrames:
    metrics_node, metrics_relation 
    '''
    df['time_stamp_start'] = pd.to_datetime(df.time_stamp_start)
    df['time_stamp_end']   = pd.to_datetime(df.time_stamp_end)

    metrics_relation = df.groupby(['url_start','url_end']).agg({'diff':['mean','max','min','sum','count'],'reload':['mean'], 'process_flag_num':['mean']})
    metrics_relation.columns = [ '_'.join(col) for col in metrics_relation.columns]
    metrics_relation = metrics_relation.rename(columns={'diff_count':'qty_rel_all','process_flag_num_mean':'process_order_mean','reload_mean':'is_reload'})
    metrics_relation = metrics_relation.reset_index()

    qty_all = df.groupby('url_start').count().rename(columns={'clientcode':'qty_all'})[['qty_all']]
    qty_unique_user = df[['clientcode','url_start']].drop_duplicates().groupby('url_start').count().rename(columns={'clientcode':'qty_unique_user'})
    qty_in = metrics_relation.reset_index().groupby('url_start').sum().rename(columns = {'qty_rel_all':'qty'})
    qty_in.columns = qty_in.columns+'_in'
    qty_out = metrics_relation.reset_index().groupby('url_end').sum().rename(columns = {'qty_rel_all':'qty'})
    qty_out.columns = qty_out.columns+'_out'

    metrics_node = pd.merge(qty_all,qty_unique_user,left_index=True, right_index=True, how = 'left')
    metrics_node = pd.merge(metrics_node,qty_in,left_index=True, right_index=True, how = 'left')
    metrics_node = pd.merge(metrics_node,qty_out,left_index=True, right_index=True, how = 'left')
    metrics_node['qty_in_out_ratio'] = metrics_node.qty_in/metrics_node.qty_out
    metrics_node = metrics_node.reset_index()

    return metrics_node, metrics_relation
 
np.random.state = 1
n, metric = 5, 'Количество переходов с предыдущей страницы на следующую'
df['clientcode'] = df['clientcode'].astype(int)
randomlist = random.sample(list(df['clientcode']), n)
df_random = df[df['clientcode'].isin(randomlist)]
sources  = list(df_random.groupby(['url_start','url_end']).size().reset_index().rename(columns={0:'cnt_open'})['url_start'])
targets = list(df_random.groupby(['url_start','url_end']).size().reset_index().rename(columns={0:'cnt_open'})['url_end'])
cnt_open = list(df_random.groupby(['url_start','url_end']).size().reset_index().rename(columns={0:'cnt_open'})['cnt_open'])
# cnt_unique =  list(df_random.groupby(['url_start','url_end']).clientcode.nunique().reset_index().rename(columns={'clientcode':'cnt_unique'})['cnt_unique'])
unique_pages = list(set(sources + targets))
numbers = range(0, len(unique_pages))
print(len(sources))
print(len(targets))
print(len(cnt_open))
d = dict(zip(unique_pages, numbers))
df_random['source_id'] = df_random['url_start'].map(d)
df_random['target_id'] = df_random['url_end'].map(d)
print(list(df_random['source_id']))
print(list(df_random['target_id']))
print(len(cnt_metric))
incoming = []
outcoming = []
for i in unique_pages:
  a = sources.count(i)
  b = targets.count(i)
  incoming.append(a)
  outcoming.append(b)

data = {'Pages': unique_pages,
        'Incoming': incoming, 
        'Outcoming': outcoming }
df_new = pd.DataFrame(data)
pages_bad= list(df_new[df_new['Incoming'] == df_new['Outcoming']]['Pages'])
df_random['GB1'] = np.where(df_random['url_start'].isin(pages_bad), 'bad', 'good')
df_random['GB2'] = df_random['GB1'].shift(-1)
df_random['url_end_shift'] = df_random['url_end'].shift(-1)
df_random['url_end_copy'] = np.where(df_random['GB2'] == 'bad', df_random['url_end_shift'], df_random['url_end'])
df_random = df_random[df_random['GB1'] != 'bad']`

# количество открытий и дальнейших переходов на другие страницы отображаются всегда
# количество выходов -- в таблиеце

table_metrics = get_metrics(df_random)[1]

if metric == 'Количество переходов с предыдущей страницы на следующую':
  cnt_metric = cnt_open

if metric == 'Количество уникальных пользователей':
  cnt_metric = cnt_unique

if metric == 'Среднее время посещения страницы':
  cnt_metric = table_metrics['diff_mean']

if metric == 'Максимальное время посещения страницы':
  cnt_metric = table_metrics['diff_max']

if metric == 'Минимальное время посещения страницы':
  cnt_metric = table_metrics['diff_min']

if metric == 'Общее время посещения страницы':
  cnt_metric = table_metrics['diff_sum']

print(len(list(df_random['source_id'])))
print(len(list(df_random['target_id'])))
print(len(cnt_metric))
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = list(d.keys()),
      color = "blue"
    ),
    link = dict(
      source = list(df_random['source_id']),
      target = list(df_random['target_id']),
      value = cnt_metric
  ))])

# fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

50
50
50
[11, 1, 9, 9, 17, 21, 11, 19, 8, 0, 23, 12, 11, 11, 11, 11, 18, 0, 15, 8, 8, 11, 23, 12, 11, 11, 11, 11, 11, 19, 3, 23, 12, 11, 23, 11, 23, 12, 11, 18, 20, 2, 10, 11, 23, 12, 9, 17, 21, 11, 16, 8, 15, 11, 8, 15, 15, 8, 0, 4, 11, 11, 15, 11, 11, 18, 8, 0, 8, 15, 11, 7, 11, 18, 0, 8, 15, 11, 3, 14, 11, 22, 22, 6, 13, 11, 8, 0, 22, 6, 13]
[1, 9, 9, 17, 21, 5, 19, 8, 0, 23, 12, 11, 11, 11, 11, 18, 0, 15, 8, 8, 11, 23, 12, 5, 11, 11, 11, 11, 19, 3, 23, 12, 5, 23, 11, 23, 12, 5, 18, 20, 2, 10, 5, 23, 12, 5, 17, 21, 5, 16, 8, 15, 11, 8, 15, 15, 8, 0, 4, 11, 11, 15, 11, 16, 18, 8, 0, 8, 15, 11, 7, 11, 18, 0, 8, 15, 11, 3, 14, 5, 22, 22, 6, 13, 5, 8, 0, 22, 6, 13, 5]
71
91
91
50


C:\Users\hp\AppData\Local\Temp\ipykernel_35380\1197727688.py:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Temp\ipykernel_35380\1197727688.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Temp\ipykernel_35380\1197727688.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [28]:
d

{'autopayments/list': 0,
 'deps': 1,
 'cardactivation/create': 2,
 'fps/create': 3,
 'complete': 4,
 'cardsmsinforming/disconnect/create': 5,
 'systemsettings/selectsysoper': 6,
 'info': 7,
 'cardsmsinforming/disconnect/confirmation': 8,
 'fps/settings/account/create': 9,
 'fps/settings/main': 10,
 'accounts': 11,
 'news': 12,
 'cardactivation/confirmation': 13,
 'main': 14,
 'paymentconfirmation': 15,
 'fps/confirmation': 16,
 'operations': 17,
 'activatesecurecodescard/create': 18,
 'cardoperations': 19,
 'informingsettings/select': 20,
 'ops': 21,
 'dcards': 22,
 'fps/settings/account/confirmation': 23,
 'paymentform': 24}

In [24]:
list(df_random['source_id'])

[14.0,
 0.0,
 nan,
 nan,
 24.0,
 15.0,
 14.0,
 nan,
 1.0,
 11.0,
 14.0,
 1.0,
 11.0,
 24.0,
 14.0,
 1.0,
 nan,
 nan,
 nan,
 14.0,
 nan,
 14.0,
 11.0,
 1.0,
 nan,
 nan,
 14.0,
 14.0,
 24.0,
 15.0,
 14.0,
 nan,
 14.0,
 24.0,
 15.0,
 14.0,
 nan,
 nan,
 nan,
 14.0,
 21.0,
 14.0,
 21.0,
 14.0,
 24.0,
 15.0,
 14.0,
 nan,
 1.0,
 24.0,
 14.0,
 21.0,
 14.0,
 21.0,
 21.0,
 21.0,
 21.0,
 21.0,
 21.0,
 11.0,
 14.0,
 19.0,
 19.0,
 11.0,
 24.0,
 15.0,
 14.0,
 14.0,
 19.0,
 19.0,
 14.0,
 11.0,
 22.0,
 14.0,
 nan,
 nan,
 nan,
 14.0,
 19.0,
 19.0,
 nan,
 22.0,
 14.0,
 14.0,
 14.0,
 19.0,
 19.0,
 24.0,
 15.0,
 22.0,
 14.0,
 19.0,
 19.0,
 11.0,
 24.0,
 15.0,
 22.0,
 19.0,
 11.0,
 22.0,
 14.0,
 22.0,
 19.0,
 19.0,
 19.0,
 22.0,
 14.0,
 nan,
 nan,
 14.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 14.0,
 1.0,
 14.0,
 14.0,
 14.0,
 22.0,
 14.0,
 nan,
 nan,
 nan,
 14.0,
 14.0,
 14.0,
 22.0,
 14.0,
 14.0,
 22.0,
 14.0,
 nan,
 14.0,
 14.0,
 22.0,
 14.0,
 nan,
 nan,
 14.0,
 22.0,
 22.0,
 14.0,
 14.0,
 22.0,
 14.0,
 na

In [10]:
super_function(df, 5, 'Количество переходов с предыдущей страницы на следующую')

148
148
81


C:\Users\hp\AppData\Local\Temp\ipykernel_35380\3869898706.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Temp\ipykernel_35380\3869898706.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hp\AppData\Local\Temp\ipykernel_35380\3869898706.py:62: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [294]:
import warnings

import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

super_function(df, 100, 'Количество уникальных пользователей')

**Сделать отдельный фильтр:**

1. Рандомное количество клиентов 
2. Циклы 
3. Перезагрузки страниц 


План: 
1. Найти слабые места с визуальной точки зрения 
2. Найти слабые места с точки зрения метрик 

Идеи: 

In [ ]:
1. Рандомное количество количество

SyntaxError: ignored